<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-201121_Regres_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [238]:
import numpy as np
# import pandas as pd
from sklearn import preprocessing

# StandardScaler

In [239]:
x=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
# x=np.array(x)
scaler = preprocessing.StandardScaler().fit(x)
print(scaler.mean_, scaler.scale_)
x_scaled = scaler.transform(x)
print (x_scaled)
x_back=scaler.inverse_transform(x_scaled)
print (x_back)
scaler.inverse_transform([[0,0,0]])
# help(scaler)

[ 0.          0.33333333 -0.16666667] [1.15470054 0.74535599 0.68718427]
[[ 1.73205081  0.89442719  0.24253563]
 [-0.8660254  -0.4472136   1.69774938]
 [ 0.         -0.4472136   0.24253563]
 [ 0.8660254   0.89442719  0.24253563]
 [-0.8660254  -1.78885438 -1.21267813]
 [-0.8660254   0.89442719 -1.21267813]]
[[ 2.  1.  0.]
 [-1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1. -1. -1.]
 [-1.  1. -1.]]


array([[ 0.        ,  0.33333333, -0.16666667]])

# Single Neuron

In [240]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ):
        """Overrides the 'linear' activation function."""
        self.activ = activ

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

In [241]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
# neu.set_activ('sigmoid')
neu.run([1,1])

5.0

## AND gate

In [242]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [243]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [244]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [245]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ) # set_activ for each neuron
    
    def set_output_activ(self, activ):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [246]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10 -10  15] sigmoid
Layer 1 Neuron 1 : [ 15  15 -10] sigmoid
Layer 2 Neuron 0 : [ 10  10 -15] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [247]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.1093778894280715
0.8928568541536401
0.7522287906530639
0.7272919934485664
0.717879460559125
0.7130288888630738
0.7100985517935621
0.7081482203089717
0.7067624976932609
0.7057304392014812

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-6.8837669  -6.96035873  0.58724611] sigmoid
Layer 1 Neuron 1 : [-4.52308149 -4.6115709  -1.44658742] sigmoid
Layer 2 Neuron 0 : [-5.65159019 -1.78043942  0.66709091] sigmoid

XOR Gate:
0 0 = 0.0354138857
0 1 = 0.6577552526
1 0 = 0.6574966580
1 1 = 0.6608391138


# Regres

In [248]:
class Regres:
    def __init__(self, layers, epochs, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        self.epochs = epochs      
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y):
        self.weight_history=[]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]])
            MSE = MSE / len(X)
            for m in range(1,len(self.regres_network.network)):
                for n in range(self.layers[m]):
                    weight_epoch.append(self.regres_network.network[m][n].weights)
            self.weight_history.append(weight_epoch)
            self.MSE_history.append(MSE)


    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

# Regres_scale

In [249]:
class Regres_scale:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        # self.epochs = epochs      
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, x, y, epochs):
        self.scaler = preprocessing.StandardScaler().fit(x)
        x_scaled=self.scaler.transform(x)
        self.epochs=epochs
        for i in range(epochs):
            MSE = 0.0
            for j in range (len(x)):
                MSE +=  self.regres_network.bp_regres(x_scaled[j],[y[j]])
            MSE = MSE / len(x)

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

    def get_standard_params(self, x):
        x2=np.array(x)
        mean=x2.mean(axis=0)
        stdev=x2.std(axis=0)
        return mean, stdev

# Example1 -- weights

In [250]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


In [251]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.99062433  0.51678285  0.2973779 ] relu
Layer 1 Neuron 1 : [-0.75152399 -0.40600667 -0.80453808] relu
Layer 2 Neuron 0 : [0.94202942 0.14148936 0.5866524 ] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.99062433  0.51678285  0.2973779 ] relu
Layer 1 Neuron 1 : [-0.75152399 -0.40600667 -0.80453808] relu
Layer 2 Neuron 0 : [0.94202942 0.14148936 0.5866524 ] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.99062433  0.51678285  0.2973779 ] linear
Layer 1 Neuron 1 : [-0.75152399 -0.40600667 -0.80453808] relu
Layer 2 Neuron 0 : [0.94202942 0.14148936 0.5866524 ] sigmoid



# Example 2 -- sigmoid, bp_classif, XOR

In [252]:
mln2 = MultiLayerNeuron(layers=[2,2,1], eta=0.5)
print (mln2.network[1][1].weights)
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))

[0.34614835 0.57653987 0.81468999]

Training Neural Network as an XOR Gate...

1.1858657791379519
1.0754728884568518
0.7478054878761056
0.11467673208204048
0.025826874992045484
0.013760798072326903
0.009244940392878634
0.006916224856129829
0.005505219908764651
0.004562435616449439

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-6.27218888  6.00720226 -3.26194334] sigmoid
Layer 1 Neuron 1 : [-5.20264924  5.37103659  2.59096667] sigmoid
Layer 2 Neuron 0 : [ 8.32263757 -8.1265695   3.83251327] sigmoid

XOR Gate:
0 0 = 0.0316655235
0 1 = 0.9715044960
1 0 = 0.9636430849
1 1 = 0.0273216180


# Example 3 -- linear activs, bp_regres

In [253]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0.,0.,0.,0.]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],[5.764286995])
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],[5.84646758])
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],[0.808633075])
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],[3.850298589])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

54.560573414122615
9.652321993077698e-13

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253294e-05] linear



# Example 4 -- sigmoid, bp_regres

In [254]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] sigmoid
Layer 1 Neuron 1 : [ 1 -2  0  1] sigmoid
Layer 2 Neuron 0 : [-1  2  3] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 19.307050477057047
MSE= 1.2477437184401223
MSE= 1.8438464279094078
MSE= 1.965891854599071


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in exp


MSE= 2.5040456749138484

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [  619.97277866 -1259.78985309   633.50387973   -13.15986688] sigmoid
Layer 1 Neuron 1 : [  614.92498529 -1231.2640606    617.66962129   -13.74953361] sigmoid
Layer 2 Neuron 0 : [0.6975949  1.30242515 2.81235505] linear



'predicted values:'

[2.812355384241326,
 3.2251834750494206,
 2.812357781046175,
 2.8123550459626667,
 2.81309616078916,
 2.8123550459626667]

# Example 5 -- various, bp_regres

In [255]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] relu
Layer 1 Neuron 1 : [ 1 -2  0  1] relu
Layer 2 Neuron 0 : [1 0 1] sigmoid
Layer 3 Neuron 0 : [-1  2] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 0.7105324736755373
MSE 200 = 0.00040110705129265404

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.17522634  0.20957589 -0.77059826  0.96170879] relu
Layer 1 Neuron 1 : [-0.19244536 -0.19119454 -0.14447966 -0.3190991 ] relu
Layer 2 Neuron 0 : [-1.02159893 -0.18087868  0.55646474] sigmoid
Layer 3 Neuron 0 : [0.6100576  0.86923583] linear



'predicted values:'

[1.0335287178503543,
 1.2547114949418627,
 1.1102562202051733,
 1.0558820236336588,
 1.0508910871711732,
 1.005270092492481]

# Example 7 -- Regres class

In [256]:
reg1=Regres(layers=[3,2,1,1], eta=0.3, epochs=4)
reg1.set_weights([[[1.,1.,0.,0.],[0.,0.,0.,0.]],[[0.,0.,0.]],[[0.,0.]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
# display ('MSE',reg1.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 9.99802753e-01  1.00026810e+00 -1.83813904e-04  4.40823369e-04] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.02773872  0.          0.00606316] sigmoid
Layer 3 Neuron 0 : [0.405996   0.81447959] linear

[1.0096496392815495, 1.0180929957513463, 1.018091754645599, 1.0124616738041174, 1.0180929957513463, 1.0180899269853907]



In [257]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
reg2=Regres_scale(layers=[3,2,1,1], eta=0.3)
reg2.set_weights([[[1.,1.,0.,0.],[0.,0.,0.,0.]],[[0.,0.,0.]],[[0.,0.]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg2.set_hidden_activ('relu')
reg2.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg2.fit(X,y,epochs=200)
reg2.printWeights()
pred=reg2.predict(X)
pred


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.90094016  1.08501405 -0.07498197  0.09504859] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.17097451  0.         -0.24461117] sigmoid
Layer 3 Neuron 0 : [0.34647447 0.87394367] linear



[0.9847645694327527,
 1.0260980418625074,
 1.0247126609442043,
 0.9966822456678671,
 1.0260980418625074,
 1.0209521040416654]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [258]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(1):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.05768057  0.24517142  0.30611063  0.52237603  0.45342795 -0.06808539] sigmoid
Layer 1 Neuron 1 : [ 0.41131404  0.07909778  0.24408963  0.85570547  0.41060858 -0.07959631] sigmoid
Layer 1 Neuron 2 : [ 0.5723622   0.91183233  0.50452277  0.83796066  0.52976436 -0.14775221] sigmoid
Layer 2 Neuron 0 : [0.60611773 0.63624137 0.50097927 1.03498116] linear

